In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/Omar')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /Omar


In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [3]:
images = []
dir = os.listdir('/Omar/My Drive/animalDataset')
inc = 1 
for sample in dir:
      if sample[0]!='.':
        img = (img_to_array(load_img('/Omar/My Drive/animalDataset/'+sample)))/255
        images.append(img)
        if inc%100==0:
          print(inc)
        inc = inc+1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


In [4]:
print(len(images))


1512


In [5]:
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/Omar/My Drive/pretrained model/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
images = np.array(images, dtype=float)


In [0]:
embed_input = Input(shape=(1000,))
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [0]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

predictions = []
X =[]
Y =[]
inc =0
for img in images:
  grayscaled_rgb = [gray2rgb(rgb2gray(img))]
  predictions.append(create_inception_embedding(grayscaled_rgb))
  lab = rgb2lab(img)
  X.append(lab[:,:,0])
  Y.append(lab[:,:,1:] / 128)
  inc+=1
  if(inc%100==0):
    print(inc)
X = np.array(X)
Y = np.array(Y)
predictions = np.array(predictions)

X = X.reshape(X.shape+(1,))
predictions = predictions.reshape((len(X),1000))

print(X.shape)
print(Y.shape)
print(predictions.shape)

In [0]:
#Train model      
tensorboard = TensorBoard(log_dir='./log')
model.compile(optimizer='adam', loss='mse')
model.fit([X,predictions],Y, callbacks=[tensorboard],validation_split=0.2, epochs=1500)

Instructions for updating:
Use tf.cast instead.
Train on 1209 samples, validate on 303 samples
Epoch 1/1500
1209/1209 [==============================] - 45s 37ms/step - loss: 0.0279 - val_loss: 0.0070
Epoch 2/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0081 - val_loss: 0.0062
Epoch 3/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0080 - val_loss: 0.0062
Epoch 4/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0078 - val_loss: 0.0062
Epoch 5/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0078 - val_loss: 0.0061
Epoch 6/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0079 - val_loss: 0.0078
Epoch 7/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0080 - val_loss: 0.0061
Epoch 8/1500
1209/1209 [==============================] - 30s 25ms/step - loss: 0.0078 - val_loss: 0.0061
Epoch 9/1500
1209/1209 [==============================] -

In [0]:
model.save('my_model_weights.h5')
